This Notebook is for my Coursera Capstone Project. I'll be using Catalonia collisions data from Kaggle ([link](https://www.kaggle.com/jordigarciacastillon/road-traffic-injuries-deaths-catalonia-201020)).

In [2]:
import pandas as pd
import numpy as np

Let's download and get familiar with the database.

In [52]:
clean_df=pd.read_csv(r"C:\Users\Iana\Dropbox\acc_catalonia_data.csv")
df=clean_df

In [53]:
df.shape

(16774, 58)

In [54]:
df.head(3)

,Any,zona,dat,via,pk,nomMun,nomCom,nomDem,F_MORTS,F_FERITS_GREUS,...,D_SUPERFICIE,D_TIPUS_VIA,D_TITULARITAT_VIA,D_TRACAT_ALTIMETRIC,D_VENT,grupDiaLab,hor,grupHor,tipAcc,tipDia
0,2010,Zona urbana,25/01/2010,SE,999999.0,CANOVES I SAMALUS,Valles Oriental,Barcelona,0,1,...,Sec i net,Via urbana( inclou carrer i carrer residencial),NaN,NaN,"Calma, vent molt suau",Feiners,2333,Nit,Col.lisió de vehicles en marxa,dill-dij
1,2010,Carretera,31/10/2010,N-240,999.0,LLEIDA,Segria,Lleida,0,1,...,Sec i net,Carretera convencional,Estatal,Pla,"Calma, vent molt suau",CapDeSetmana,1,Nit,Sortida de la calcada sense especificar,dg
2,2010,Carretera,17/05/2010,N-II,7087.0,FORNELLS DE LA SELVA,Girones,Girona,1,0,...,Sec i net,Carretera convencional,Estatal,Rampa o pendent,"Calma, vent molt suau",Feiners,1527,Tarda,Col.lisió de vehicles en marxa,dill-dij


It'd be better if we translate columns from Catalan to English.

In [95]:
df.columns=['YEAR', 'AREA', 'DATE', 'ROAD', 'KM', 'CITY', 'COUNTY', 'DEMARCATION',
       'N_DEATHS', 'N_MAJ_INJURIES', 'N_MIN_INJURIES', 'N_VICTIMS',
       'N_VEHICLES', 'N_PEDESTRIANS',
       'N_BICYCLES', 'N_MOPEDS',
       'N_MOTORCYCLES', 'N_LIGHT_VEHICLES',
       'N_HEAVY_VEHICLES', 'N_OTHER_TYPES',
       'N_UNKNOWN_TYPE', 'ALLOWED_SPEED', 'LEAK',
       'FOG', 'TERRAIN', 'SPECIAL_LANE',
       'SPECIAL_MEASURES', 'CLIMATOLOGY', 'SPECIAL_FUNCTION',
       'SEVERITY', 'FOG_INFLUENCE', 'TERRAIN_INFLUENCE',
       'TRAFFIC_INFLUENCE', 'WEATHER_INFLUENCE', 'WIND_INFLUENCE',
       'LUMINOSITY_INFLUENCE', 'SPECIAL_MEASURES_INFLUENCE', 'OBJECT_INFLUENCE',
       'FURROWS_INFLUENCE', 'VISIBILITY_INFLUENCE', 'INTERSECTION',
       'SPEED_LIMIT_DISPLAY', 'LIGHT_CONDITIONS', 'REGULATION_OF_PRIORITY',
       'TRACK_TRAJECTORIES', 'ACC_CLASSIFICATION', 'TYPE_OF_SECTION', 'AREA_CLASSIFICATION',
       'ROAD_STATE', 'ROAD_TYPE', 'ROAD_OWNERSHIP',
       'ALTIMETRIC_LAYOUT', 'WIND_CLASSIFICATION', 'WORKING_DAY', 'TIME', 'TIME_SLOT',
       'ACC_TYPE', 'WEEKDAY']
df.head()

,YEAR,AREA,DATE,ROAD,KM,CITY,COUNTY,DEMARCATION,N_DEATHS,N_MAJ_INJURIES,...,ROAD_STATE,ROAD_TYPE,ROAD_OWNERSHIP,ALTIMETRIC_LAYOUT,WIND_CLASSIFICATION,WORKING_DAY,TIME,TIME_SLOT,ACC_TYPE,WEEKDAY
0,2010,Zona urbana,25/01/2010,SE,999999.0,CANOVES I SAMALUS,Valles Oriental,Barcelona,0,1,...,Sec i net,Via urbana( inclou carrer i carrer residencial),NaN,NaN,"Calma, vent molt suau",Feiners,2333,Nit,Col.lisió de vehicles en marxa,dill-dij
1,2010,Carretera,31/10/2010,N-240,999.0,LLEIDA,Segria,Lleida,0,1,...,Sec i net,Carretera convencional,Estatal,Pla,"Calma, vent molt suau",CapDeSetmana,1,Nit,Sortida de la calcada sense especificar,dg
2,2010,Carretera,17/05/2010,N-II,7087.0,FORNELLS DE LA SELVA,Girones,Girona,1,0,...,Sec i net,Carretera convencional,Estatal,Rampa o pendent,"Calma, vent molt suau",Feiners,1527,Tarda,Col.lisió de vehicles en marxa,dill-dij
3,2010,Zona urbana,21/08/2010,SE,999999.0,BARCELONA,Barcelones,Barcelona,0,2,...,Sec i net,Via urbana( inclou carrer i carrer residencial),NaN,NaN,"Calma, vent molt suau",CapDeSetmana,223,Nit,Col.lisió de vehicles en marxa,dis
4,2010,Zona urbana,07/05/2010,SE,999999.0,BADALONA,Barcelones,Barcelona,0,1,...,Sec i net,Via urbana( inclou carrer i carrer residencial),NaN,NaN,"Calma, vent molt suau",CapDeSetmana,1745,Tarda,Bolcada a la calcada,div


Translating rows and cleaning up!

In [110]:
df=df.replace({'No':'No','Si':'Yes','Sense Especificar':'Not specified',"No n'hi ha":"No"})
df["AREA"]=df["AREA"].replace({'Zona urbana': 'Urban area','Carretera': 'Road'})
df["TERRAIN"]=df["TERRAIN"].replace({'Desmunt':'Disassembled','A nivell':'Plane','Sense Especificar':'Not specified','Mixt':'Mixed','Terraplé':'Embankment'})
df["SPECIAL_LANE"]=df["SPECIAL_LANE"].replace({"No n'hi ha":"No", 'Carril habilitat en sentit contrari habitual':'Lane enabled in the opposite direction than usual','Habilitació voral/carril addicional':'Additional curb / lane clearance','Carril lent':'Slow lane', "Carril d'alentiment":'Slowdown lane','Altres':'Others','Carril bus':'Bus lane','Carril central':'Central lane','Carril bici':'Cycling lane','Carril avançament':'Overtaking lane','Carril reversible':'Reversible lane','Carril acceleració':'Acceleration lane'})
df["SPECIAL_MEASURES"]=df["SPECIAL_MEASURES"].replace({"No n'hi ha":"No",'Obres':'Works','Serveis de neteja o manteniment':'Cleaning or maintenance services','Esdeveniment extraordinari':'Extraordinary event','Accident trànsit anterior':'Previous traffic accident','Cons':'Built','Control policial':'Police control'})
df["CLIMATOLOGY"]=df["CLIMATOLOGY"].replace({'Bon temps':'Good weather','Pluja forta':'Heavy rain','Pluja dèbil':'Light rain','Nevant':'Snowing','Calamarsa':'Hail'})
df["SPECIAL_FUNCTION"]=df["SPECIAL_FUNCTION"].replace({'Sense funció especial':'No special function','Variant':'Alternative','Travessera':'Crossing','Ronda, cinturo o circumval.lacio':'Round, waist or circumvallation'})
df["SEVERITY"]=df["SEVERITY"].replace({'Accident greu':'Serious accident','Accident mortal':'Fatal accident'})
df["INTERSECTION"]=df["INTERSECTION"].replace({'Arribant o eixint intersecció fins 50m':'Arriving or leaving intersection up to 50m','Dintre intersecció':'Inside intersection','En secció':'In section'})
df["SPEED_LIMIT_DISPLAY"]=df["SPEED_LIMIT_DISPLAY"].replace({'Genérica via':'Generic way','Senyal velocitat':'Speed signal'})
df["LIGHT_CONDITIONS"]=df["LIGHT_CONDITIONS"].replace({'De nit, il·luminació artificial suficient':'At night, enough artificial lighting','De dia, dia clar':'Day, clear day','De nit, sense llum artificial':'At night, without artificial light','De dia, dia fosc':'Day, dark day','Alba o capvespre':'Dawn or dusk','De nit, il·luminació artificial insuficient':'At night, insufficient artificial lighting'})
df["REGULATION_OF_PRIORITY"]=df["REGULATION_OF_PRIORITY"].replace({'Sols norma prioritat de pas':'Only standart way','Senyal Stop o cedeix pas':'Stop signal or give way','Semàfor':'Traffic light','Sols marques viàries (inclou pas vianants)':'Road markings only (includes pedestrian crossing)','Persona autoritzada':'Authorized person','Altres':'Others'})
df["TRACK_TRAJECTORIES"]=df["TRACK_TRAJECTORIES"].replace({'Un sol sentit':'One way','Doble sentit':'Double way'})
df["ACC_CLASSIFICATION"]=df["ACC_CLASSIFICATION"].replace({'Encalç':'Chase','Resta sortides de via':'Other road departures','Col·lisió frontal':'Front collision','Envestida (frontal lateral)':'Onslaught (front side)','Caiguda en la via':'Fall on the road','Atropellament':'Run over','Fregament o col·lisió lateral':'Friction or side collision','Xoc contra objecte/obstacle sense sortida prèvia de via':'Collision against object / obstacle without previous derailment','Altres':'Others','Sortida de via amb xoc o col·lisió':'Derailment with collision or clash','Sortida de via amb bolcada':'Derailment with dump','Sortida de via amb atropellament':'Derailment with collision','Xoc amb animal a la calçada':'Collision with animal on the road'})
df["TYPE_OF_SECTION"]=df["TYPE_OF_SECTION"].replace({'Intersecció en T o Y':'T or Y intersection','Giratòria':'Rotary','Encreuament o intersecció en X o +':'Crossing or intersection at X or +',"Enllaç d'entrada o eixida":"Inbound or outbound link",'Pas a nivell':'Level crossing'})
df["AREA_CLASSIFICATION"]=df["AREA_CLASSIFICATION"].replace({'Zona urbana': 'Urban area','Carretera': 'Road','Travessera':'Crossing'})
df["ROAD_STATE"]=df["ROAD_STATE"].replace({'Sec i net':'Dry and clean','Relliscós':'Slippery','Mullat':'Wet','Inundat':'Flooded','Nevat':'Snowy','Gelat':'Icy'})
df["ROAD_TYPE"]=df["ROAD_TYPE"].replace({'Via urbana( inclou carrer i carrer residencial)':'Urban road (includes street and residential street)','Carretera convencional':'Conventional road','Altres':'Others','Autopista':'Highway', 'Autovia':'Freeway', 'Camí rural/pista forestal':'Rural road / forest track'})
df['ROAD_OWNERSHIP']=df['ROAD_OWNERSHIP'].replace({'Estatal':'State','Municipal':'Municipal','Autonòmica':'Autonomous','Provincial':'Provincial','Altres':'Others'})
df["ALTIMETRIC_LAYOUT"]=df["ALTIMETRIC_LAYOUT"].replace({'Pla':'Flat','Rampa o pendent':'Ramp or slope','Canvi rasant':'Ground change', 'Gual':'Ford'})
df["WIND_CLASSIFICATION"]=df["WIND_CLASSIFICATION"].replace({'Calma, vent molt suau':'Calm, very gentle wind','Vent moderat':'Moderate wind','Vent fort':'Strong wind'})
df["WORKING_DAY"]=df["WORKING_DAY"].replace({'Feiners':'Workdays','CapDeSetmana':'Weekend'})
df["TIME_SLOT"]=df["TIME_SLOT"].replace({'Nit':'Night', 'Tarda':'Afternoon','Matí':'Morning'})
df["ACC_TYPE"]=df["ACC_TYPE"].replace({'Col.lisió de vehicles en marxa':'Collision of vehicles in motion','Sortida de la calcada sense especificar':'Unspecified track exit','Bolcada a la calcada':'Dump on the track', 'Atropellament':'Run over',"Col.lisió d'un vehicle contra un obstacle de la calcada":"Collision of a vehicle against a road obstacle",'Altres':'Others'})
df["WEEKDAY"]=df["WEEKDAY"].replace({'dill-dij':'mon-thu','dg':'sun','dis':'sat','div':'fri'})

In [113]:
df.head()

,YEAR,AREA,DATE,ROAD,KM,CITY,COUNTY,DEMARCATION,N_DEATHS,N_MAJ_INJURIES,...,ROAD_STATE,ROAD_TYPE,ROAD_OWNERSHIP,ALTIMETRIC_LAYOUT,WIND_CLASSIFICATION,WORKING_DAY,TIME,TIME_SLOT,ACC_TYPE,WEEKDAY
0,2010,Urban area,25/01/2010,SE,999999.0,CANOVES I SAMALUS,Valles Oriental,Barcelona,0,1,...,Dry and clean,Urban road (includes street and residential st...,NaN,NaN,"Calm, very gentle wind",Workdays,2333,Night,Collision of vehicles in motion,mon-thu
1,2010,Road,31/10/2010,N-240,999.0,LLEIDA,Segria,Lleida,0,1,...,Dry and clean,Conventional road,State,Flat,"Calm, very gentle wind",Weekend,1,Night,Unspecified track exit,sun
2,2010,Road,17/05/2010,N-II,7087.0,FORNELLS DE LA SELVA,Girones,Girona,1,0,...,Dry and clean,Conventional road,State,Ramp or slope,"Calm, very gentle wind",Workdays,1527,Afternoon,Collision of vehicles in motion,mon-thu
3,2010,Urban area,21/08/2010,SE,999999.0,BARCELONA,Barcelones,Barcelona,0,2,...,Dry and clean,Urban road (includes street and residential st...,NaN,NaN,"Calm, very gentle wind",Weekend,223,Night,Collision of vehicles in motion,sat
4,2010,Urban area,07/05/2010,SE,999999.0,BADALONA,Barcelones,Barcelona,0,1,...,Dry and clean,Urban road (includes street and residential st...,NaN,NaN,"Calm, very gentle wind",Weekend,1745,Afternoon,Dump on the track,fri
